# Import libraries

In [1]:
import os, glob, csv, sys, shutil, math, random
from os import path
import pandas as pd
import numpy as np

# Define directories

In [2]:
userhome = os.path.expanduser('~')
file_dir = userhome + r'/mining-forum/analyze/csv_files/links_list/cleaned/'
summ_dir = userhome + r'/mining-forum/analyze/csv_files/summarize/'

# Collect files

In [3]:
files = []
for file in glob.iglob(file_dir + '*'):
    files.append(file)
files.sort()
files

['/Users/YusufNugroho/mining-forum/analyze/csv_files/links_list/cleaned/links_without_bot_authors_forum_12.csv',
 '/Users/YusufNugroho/mining-forum/analyze/csv_files/links_list/cleaned/links_without_bot_authors_forum_13.csv',
 '/Users/YusufNugroho/mining-forum/analyze/csv_files/links_list/cleaned/links_without_bot_authors_forum_14.csv',
 '/Users/YusufNugroho/mining-forum/analyze/csv_files/links_list/cleaned/links_without_bot_authors_forum_15.csv',
 '/Users/YusufNugroho/mining-forum/analyze/csv_files/links_list/cleaned/links_without_bot_authors_forum_16.csv',
 '/Users/YusufNugroho/mining-forum/analyze/csv_files/links_list/cleaned/links_without_bot_authors_forum_17.csv',
 '/Users/YusufNugroho/mining-forum/analyze/csv_files/links_list/cleaned/links_without_bot_authors_forum_18.csv',
 '/Users/YusufNugroho/mining-forum/analyze/csv_files/links_list/cleaned/links_without_bot_authors_forum_19.csv',
 '/Users/YusufNugroho/mining-forum/analyze/csv_files/links_list/cleaned/links_without_bot_author

In [4]:
len(files)

9

# Joining all csv files

In [5]:
result = pd.DataFrame([])
for i, fname in enumerate(files):
    ff = pd.read_csv(fname, skiprows=0, usecols=[0,1,2,3,4,5,6])
    result = result.append(ff)
result.to_csv(summ_dir + 'all_joined_links.csv', index=False)
print ("CSV file has been created")

CSV file has been created


In [6]:
result = pd.read_csv(summ_dir + 'all_joined_links.csv')
result = result.drop_duplicates(subset=['file_number','message_id','topic','author','link'], keep='first')

In [7]:
result.to_csv(summ_dir + 'all_joined_links.csv', index=False)
result

,file_number,message_id,topic,author,link,domain,top_level_domain
0,10,message #23,Some thoughts,Jody Garnett,http://udig.refractions.net/,udig.refractions.net,net
1,100,message #255,Webex: Target Management Meeting,Doug Gaff,https://intercall.webex.com/intercall/j.php?ED...,intercall.webex.com,com
2,1000,message #2645,Re: GMF errors,Richard Gronback,http://download.eclipse.org/modeling/gmf/updat...,download.eclipse.org,org
3,10000,message #30746,[BUILD] R1.0-I,Alex Panchenko,http://download.eclipse.org/technology/dltk/do...,download.eclipse.org,org
4,10001,message #30751,Stumped by installation.,Eclipse User,http://www.eclipsefaq.org/chris/faq/html/faq25...,www.eclipsefaq.org,org
5,10001,message #30751,Stumped by installation.,Eclipse User,http://eclipsefaq.org,eclipsefaq.org,org
6,10003,message #30761,Problems with too many clicks on buttons,Roberto Sanchez,https://bugs.eclipse.org/bugs/show_bug.cgi?id=...,bugs.eclipse.org,org
7,10004,message #30762,How to get a classloader for the project in a ...,Eclipse User,http://eclipsewiki.swiki.net/114#augmentedClas...,eclipsewiki.swiki.net,net
8,10006,message #30770,New STEM Design Document Wiki,Daniel Ford,http://wiki.eclipse.org/STEM_Design_Document#C...,wiki.eclipse.org,org
9,10008,message #30789,IAC question,Eclipse User,http://www.eclipse.org/tptp/platform/documents...,www.eclipse.org,org


# Apply random sampling

Determine the size of sample

In [8]:
pop = len(result)
Z = 1.96          #Z value (e.g., 1.96 for 95% confidence level)
p = 0.5           #percentage picking a choice, expressed as decimal (0.5 used for sample size needed)
c = 0.05          #confidence interval, expressed as decimal (e.g., 0.04 = +-4)

ss = (math.pow(Z,2) * p * (1-p)) / math.pow(c,2)
new_ss = ss / (1 + ((ss-1)/pop))
print ("Sample size needed : %i" % round(new_ss))

Sample size needed : 383


In [9]:
filename = summ_dir + 'all_joined_links.csv'
nlinesrandomsample = round(new_ss)
lines2skip = np.random.choice(np.arange(1,pop+1), (pop-nlinesrandomsample), replace=False)
df = pd.read_csv(filename, skiprows=lines2skip)
df.to_csv(summ_dir + 'links_of_randomsampling.csv')
df

,file_number,message_id,topic,author,link,domain,top_level_domain
0,10442,message #32194,[Announce] EMFT VALIDATION 1.0.0RC1 is available,Eclipse User,http://www.eclipse.org/emft/news/release-notes...,www.eclipse.org,org
1,11097,message #34458,[buckminster-dev] +1 for Ed Merks on tools.buc...,Karel Brezina,http://portal.eclipse.org/,portal.eclipse.org,org
2,11563,message #35905,Status update on some new features.,Ketan Padegaonkar,http://twitter.com/ketanpkr,twitter.com,com
3,12150,message #37846,What causes this Mylyn error?,Kaj Kandler,http://plan-b-for-openoffice.org/,plan-b-for-openoffice.org,org
4,13069,message #41104,install document,Ketan Padegaonkar,http://download.eclipse.org/technology/swtbot/...,download.eclipse.org,org
5,13404,message #42255,[ATL] problem with programmatically transforma...,Eclipse User,http://www.eclipse.org/emf/2002/Ecore#//EString,www.eclipse.org,org
6,13928,message #44090,PDE-Build against a target platform,Andrew Niefer,http://help.eclipse.org/ganymede/index.jsp?top...,help.eclipse.org,org
7,15163,message #48450,Re: EMF/SDO/XSD Update Site,Eclipse User,http://dev.eclipse.org/viewcvs/indextools.cgi/...,dev.eclipse.org,org
8,15494,message #49527,To clear up some confusion regarding the commi...,Eclipse User,http://bugs.eclipse.org/224853,bugs.eclipse.org,org
9,15719,message #50285,Development build containing AspectJ 5 M2 cand...,Eclipse User,http://www.eclipse.org/ajdt,www.eclipse.org,org


# Capture domain plus path 1

In [55]:
import re

In [67]:
new_data = []
for n in range(0,len(result)):
    url = result.iloc[n][4]
    
    sys.stdout.write('\rExtracting : %i' %n + '/%i' %(len(result)))
    sys.stdout.flush()
    
    reghttp = re.search("(http.?://)", url)
    reghttp = reghttp.group()
    try:       
        regpath = re.search("http.?:\/\/(\w*.+?\/\w*[a-zA-Z0-9._#:%@&?=-]+\w*)\/?", url)
        regpath = regpath.group(1)
    except:
        regpath = result.iloc[n][5]

    
    temp = [result.iloc[n][0], result.iloc[n][1], result.iloc[n][2], result.iloc[n][3], result.iloc[n][4], regpath, result.iloc[n][5], result.iloc[n][6]]
    new_data.append(temp)
print ("\nProcess is finished")

Extracting : 216863/216864Process is finished


In [68]:
new_data

[[10,
  'message #23',
  'Some thoughts',
  'Jody Garnett',
  'http://udig.refractions.net/',
  'udig.refractions.net',
  'udig.refractions.net',
  'net'],
 [100,
  'message #255',
  'Webex: Target Management Meeting',
  'Doug Gaff',
  'https://intercall.webex.com/intercall/j.php?ED=88273012&UID=33408652',
  'intercall.webex.com/intercall',
  'intercall.webex.com',
  'com'],
 [1000,
  'message #2645',
  'Re: GMF errors',
  'Richard Gronback',
  'http://download.eclipse.org/modeling/gmf/update-site/releases/site.xml',
  'download.eclipse.org/modeling',
  'download.eclipse.org',
  'org'],
 [10000,
  'message #30746',
  '[BUILD] R1.0-I',
  'Alex Panchenko',
  'http://download.eclipse.org/technology/dltk/downloads/drops/R1.0/I-I200808271527-200808271527/',
  'download.eclipse.org/technology',
  'download.eclipse.org',
  'org'],
 [10001,
  'message #30751',
  'Stumped by installation.',
  'Eclipse User',
  'http://www.eclipsefaq.org/chris/faq/html/faq25.html',
  'www.eclipsefaq.org/chris',


In [69]:
with open(summ_dir + 'all_joined_links_with_path.csv','w') as mycsv:
    head = ['file_number','message_id','topic','author','link','link_path','domain','top_level_domain']
    writers = csv.writer(mycsv, delimiter = ',')
    writers.writerow(head)
    for item in new_data:
        writers.writerow(item)
print("CSV file has been created")

CSV file has been created
